# Manipulating Data in DataFrames HW


#### Let's get started applying what we learned in the lecure!

I've provided several questions below to help test and expand you knowledge from the code along lecture. So let's see what you've got!

First create your spark instance as we need to do at the start of every project.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
path = "drive/MyDrive/5. Spark/spark-scripts/section2/Datasets/"

Mounted at /content/drive


## Read in our Republican vs. Democrats Tweet DataFrame

Attached to the lecture

In [ ]:
!pip install pyspark
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('manip').getOrCreate()
spark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 17.5 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=3d8abd8c2bff26771b82203d2bd99625791d50c1ae7521d97b1bf6955cc489de
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [ ]:
df = spark.read.csv(path+'Rep_vs_Dem_tweets.csv', inferSchema=True, header=True)

## About this dataframe

Extracted tweets from all of the representatives (latest 200 as of May 17th 2018)

**Source:** https://www.kaggle.com/kapastor/democratvsrepublicantweets#ExtractedTweets.csv

Use either .show() or .toPandas() check out the first view rows of the dataframe to get an idea of what we are working with.

In [ ]:
df.limit(10).toPandas()

,Party,Handle,Tweet
0,Democrat,RepDarrenSoto,"Today, Senate Dems vote to #SaveTheInternet. Proud to support similar #NetNeutrality legislation here in the House… https://t.co/n3tggDLU1L"
1,Democrat,RepDarrenSoto,RT @WinterHavenSun: Winter Haven resident / Alta Vista teacher is one of several recognized by @RepDarrenSoto for National Teacher Apprecia…
2,Democrat,RepDarrenSoto,RT @NBCLatino: .@RepDarrenSoto noted that Hurricane Maria has left approximately $90 billion in damages.
3,"Congress has allocated about $18…""",None,None
4,Democrat,RepDarrenSoto,RT @NALCABPolicy: Meeting with @RepDarrenSoto . Thanks for taking the time to meet with @LatinoLeader ED Marucci Guzman. #NALCABPolicy2018.…
5,Democrat,RepDarrenSoto,RT @Vegalteno: Hurricane season starts on June 1st; Puerto Rico’s readiness...well 🤦🏼‍♂️😡😩@Pwr4PuertoRico @RepDarrenSoto @EspaillatNY
6,Democrat,RepDarrenSoto,RT @EmgageActionFL: Thank you to all who came out to our Orlando gala! It was a successful night that would not have been possible without…
7,Democrat,RepDarrenSoto,"Hurricane Maria left approx $90 billion in damages, yet only $1 billion was allocated for rebuilding grid. No surpr… https://t.co/2kU8BcKwUh"
8,Democrat,RepDarrenSoto,RT @Tharryry: I am delighted that @RepDarrenSoto will be voting for the CRA to overrule the FCC and save our #NetNeutrality rules. Find out…
9,Democrat,RepDarrenSoto,RT @HispanicCaucus: Trump's anti-immigrant policies are hurting small businesses across the country that can’t find Americans willing to do…


**Prevent Truncation of view**

If the view you produced above truncated some of the longer tweets, see if you can prevent that so you can read the whole tweet.

In [ ]:
import pandas as pd
pd.set_option('display.max_columns',None)
pd.set_option('display.max_colwidth',None)
pd.set_option('display.max_rows',None)
df.limit(5).toPandas()

,Party,Handle,Tweet
0,Democrat,RepDarrenSoto,"Today, Senate Dems vote to #SaveTheInternet. Proud to support similar #NetNeutrality legislation here in the House… https://t.co/n3tggDLU1L"
1,Democrat,RepDarrenSoto,RT @WinterHavenSun: Winter Haven resident / Alta Vista teacher is one of several recognized by @RepDarrenSoto for National Teacher Apprecia…
2,Democrat,RepDarrenSoto,RT @NBCLatino: .@RepDarrenSoto noted that Hurricane Maria has left approximately $90 billion in damages.
3,"Congress has allocated about $18…""",None,None
4,Democrat,RepDarrenSoto,RT @NALCABPolicy: Meeting with @RepDarrenSoto . Thanks for taking the time to meet with @LatinoLeader ED Marucci Guzman. #NALCABPolicy2018.…


**Print Schema**

First, check the schema to make sure the datatypes are accurate. 

In [ ]:
df.printSchema()

root
 |-- Party: string (nullable = true)
 |-- Handle: string (nullable = true)
 |-- Tweet: string (nullable = true)



## 1. Can you identify any tweet that mentions the handle @LatinoLeader using regexp_extract?

It doesn't matter how you identify the row, any identifier will do. You can test your script on row 5 from this dataset. That row contains @LatinoLeader. 

In [ ]:
from pyspark.sql.functions import regexp_replace, regexp_extract
df.withColumn("check",regexp_extract('Tweet','LatinoLeader', 0)).select('Party','Handle','Tweet').filter("check == 'LatinoLeader' ").toPandas()
# It doesn't work when I add the '@' symbol !!

,Party,Handle,Tweet
0,Democrat,RepDarrenSoto,RT @NALCABPolicy: Meeting with @RepDarrenSoto . Thanks for taking the time to meet with @LatinoLeader ED Marucci Guzman. #NALCABPolicy2018.…


## 2. Replace any value other than 'Democrate' or 'Republican' with 'Other' in the Party column.

We can see from the output below, that there are several other values other than 'Democrate' or 'Republican' in the Part column. We are assuming that this is dirty data that needs to be cleaned up.

In [ ]:
df.select('Party').filter("Party == 'Republican'").distinct().toPandas()

,Party
0,Republican


In [ ]:
df.select('Party').filter("Party == 'Democrate'").distinct().toPandas()

,Party


In [ ]:
from pyspark.sql.functions import when

df = df.withColumn('Party', when(df.Party == 'Democrat', 'Democrate'  ).when(df.Party == 'Republican', 'Republican').otherwise('Other'))


In [ ]:
df.limit(5).toPandas()

,Party,Handle,Tweet
0,Democrate,RepDarrenSoto,"Today, Senate Dems vote to #SaveTheInternet. Proud to support similar #NetNeutrality legislation here in the House… https://t.co/n3tggDLU1L"
1,Democrate,RepDarrenSoto,RT @WinterHavenSun: Winter Haven resident / Alta Vista teacher is one of several recognized by @RepDarrenSoto for National Teacher Apprecia…
2,Democrate,RepDarrenSoto,RT @NBCLatino: .@RepDarrenSoto noted that Hurricane Maria has left approximately $90 billion in damages.
3,Other,None,None
4,Democrate,RepDarrenSoto,RT @NALCABPolicy: Meeting with @RepDarrenSoto . Thanks for taking the time to meet with @LatinoLeader ED Marucci Guzman. #NALCABPolicy2018.…


## 3. Delete all embedded links (ie. "https:....)

For example see the first row in the tweets dataframe. 

*Note: this may require an google search :)*

In [ ]:
df.limit(5).toPandas()

,Party,Handle,Tweet
0,Democrate,RepDarrenSoto,"Today, Senate Dems vote to #SaveTheInternet. Proud to support similar #NetNeutrality legislation here in the House… https://t.co/n3tggDLU1L"
1,Democrate,RepDarrenSoto,RT @WinterHavenSun: Winter Haven resident / Alta Vista teacher is one of several recognized by @RepDarrenSoto for National Teacher Apprecia…
2,Democrate,RepDarrenSoto,RT @NBCLatino: .@RepDarrenSoto noted that Hurricane Maria has left approximately $90 billion in damages.
3,Other,None,None
4,Democrate,RepDarrenSoto,RT @NALCABPolicy: Meeting with @RepDarrenSoto . Thanks for taking the time to meet with @LatinoLeader ED Marucci Guzman. #NALCABPolicy2018.…


In [ ]:
df = df.withColumn('Tweet',regexp_replace(df.Tweet,'https?://[^\s]+',''))

In [ ]:
df.limit(5).toPandas()

,Party,Handle,Tweet
0,Democrate,RepDarrenSoto,"Today, Senate Dems vote to #SaveTheInternet. Proud to support similar #NetNeutrality legislation here in the House…"
1,Democrate,RepDarrenSoto,RT @WinterHavenSun: Winter Haven resident / Alta Vista teacher is one of several recognized by @RepDarrenSoto for National Teacher Apprecia…
2,Democrate,RepDarrenSoto,RT @NBCLatino: .@RepDarrenSoto noted that Hurricane Maria has left approximately $90 billion in damages.
3,Other,None,None
4,Democrate,RepDarrenSoto,RT @NALCABPolicy: Meeting with @RepDarrenSoto . Thanks for taking the time to meet with @LatinoLeader ED Marucci Guzman. #NALCABPolicy2018.…


## 4. Remove any leading or trailing white space in the tweet column

In [ ]:
from pyspark.sql.functions import *

df = df.withColumn('Tweet',trim(df.Tweet))

In [ ]:
df.limit(5).toPandas()

,Party,Handle,Tweet
0,Democrate,RepDarrenSoto,"Today, Senate Dems vote to #SaveTheInternet. Proud to support similar #NetNeutrality legislation here in the House…"
1,Democrate,RepDarrenSoto,RT @WinterHavenSun: Winter Haven resident / Alta Vista teacher is one of several recognized by @RepDarrenSoto for National Teacher Apprecia…
2,Democrate,RepDarrenSoto,RT @NBCLatino: .@RepDarrenSoto noted that Hurricane Maria has left approximately $90 billion in damages.
3,Other,None,None
4,Democrate,RepDarrenSoto,RT @NALCABPolicy: Meeting with @RepDarrenSoto . Thanks for taking the time to meet with @LatinoLeader ED Marucci Guzman. #NALCABPolicy2018.…


## 5. Rename the 'Party' column to 'Dem_Rep'

No real reason here :) just wanted you to get practice doing this. 

In [ ]:
df.withColumnRenamed('Party','Dem_Rep').limit(5).toPandas()

,Dem_Rep,Handle,Tweet
0,Democrate,RepDarrenSoto,"Today, Senate Dems vote to #SaveTheInternet. Proud to support similar #NetNeutrality legislation here in the House…"
1,Democrate,RepDarrenSoto,RT @WinterHavenSun: Winter Haven resident / Alta Vista teacher is one of several recognized by @RepDarrenSoto for National Teacher Apprecia…
2,Democrate,RepDarrenSoto,RT @NBCLatino: .@RepDarrenSoto noted that Hurricane Maria has left approximately $90 billion in damages.
3,Other,None,None
4,Democrate,RepDarrenSoto,RT @NALCABPolicy: Meeting with @RepDarrenSoto . Thanks for taking the time to meet with @LatinoLeader ED Marucci Guzman. #NALCABPolicy2018.…


## 6. Concatenate the Party and Handle columns

Silly yes... but good practice.

pyspark.sql.functions.concat_ws(sep, *cols)[source] <br>
Concatenates multiple input string columns together into a single string column, using the given separator.

In [ ]:
df.select(concat_ws(' ',df.Party,df.Handle).alias('Concatenated')).show(5,False)

+-----------------------+
|Concatenated           |
+-----------------------+
|Democrate RepDarrenSoto|
|Democrate RepDarrenSoto|
|Democrate RepDarrenSoto|
|Other                  |
|Democrate RepDarrenSoto|
+-----------------------+
only showing top 5 rows



## Challenge Question

Let's image that we want to analyze the hashtags that are used in these tweets. Can you extract all the hashtags you see?

# Let's create our own dataset to work with real dates

This is a dataset of patient visits from a medical office. It contains the patients first and last names, date of birth, and the dates of their first 3 visits. 

In [ ]:
from pyspark.sql.types import *

md_office = [('Mohammed','Alfasy','1987-4-8','2016-1-7','2017-2-3','2018-3-2') \
            ,('Marcy','Wellmaker','1986-4-8','2015-1-7','2017-1-3','2018-1-2') \
            ,('Ginny','Ginger','1986-7-10','2014-8-7','2015-2-3','2016-3-2') \
            ,('Vijay','Doberson','1988-5-2','2016-1-7','2018-2-3','2018-3-2') \
            ,('Orhan','Gelicek','1987-5-11','2016-5-7','2017-1-3','2018-9-2') \
            ,('Sarah','Jones','1956-7-6','2016-4-7','2017-8-3','2018-10-2') \
            ,('John','Johnson','2017-10-12','2018-1-2','2018-10-3','2018-3-2') ]

df = spark.createDataFrame(md_office,['first_name','last_name','dob','visit1','visit2','visit3']) # schema=final_struc

# Check to make sure it worked
df.show()
print(df.printSchema())

+----------+---------+----------+--------+---------+---------+
|first_name|last_name|       dob|  visit1|   visit2|   visit3|
+----------+---------+----------+--------+---------+---------+
|  Mohammed|   Alfasy|  1987-4-8|2016-1-7| 2017-2-3| 2018-3-2|
|     Marcy|Wellmaker|  1986-4-8|2015-1-7| 2017-1-3| 2018-1-2|
|     Ginny|   Ginger| 1986-7-10|2014-8-7| 2015-2-3| 2016-3-2|
|     Vijay| Doberson|  1988-5-2|2016-1-7| 2018-2-3| 2018-3-2|
|     Orhan|  Gelicek| 1987-5-11|2016-5-7| 2017-1-3| 2018-9-2|
|     Sarah|    Jones|  1956-7-6|2016-4-7| 2017-8-3|2018-10-2|
|      John|  Johnson|2017-10-12|2018-1-2|2018-10-3| 2018-3-2|
+----------+---------+----------+--------+---------+---------+

root
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- dob: string (nullable = true)
 |-- visit1: string (nullable = true)
 |-- visit2: string (nullable = true)
 |-- visit3: string (nullable = true)

None


Oh no! The dates are still stored as text... let's try converting them again and see if we have any issues this time.

In [ ]:
df = df.withColumn('visit1',to_date(df.visit1,'yyyy-d-M')) \
       .withColumn('visit2',to_date(df.visit2,'yyyy-d-M')) \
       .withColumn('visit3',to_date(df.visit3,'yyyy-d-M'))

In [ ]:
df.show(3)

+----------+---------+---------+----------+----------+----------+
|first_name|last_name|      dob|    visit1|    visit2|    visit3|
+----------+---------+---------+----------+----------+----------+
|  Mohammed|   Alfasy| 1987-4-8|2016-07-01|2017-03-02|2018-02-03|
|     Marcy|Wellmaker| 1986-4-8|2015-07-01|2017-03-01|2018-02-01|
|     Ginny|   Ginger|1986-7-10|2014-07-08|2015-03-02|2016-02-03|
+----------+---------+---------+----------+----------+----------+
only showing top 3 rows



In [ ]:
df.printSchema()

root
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- dob: string (nullable = true)
 |-- visit1: date (nullable = true)
 |-- visit2: date (nullable = true)
 |-- visit3: date (nullable = true)



## 7. Can you calculate a variable showing the length of time between patient visits?

Compare visit1 to visit2 and visit2 to visit3 for all patients and see what the average length of time is between visits. Create an alias for it as well. 

In [ ]:
df = df.withColumn('diff 1&2',datediff(df.visit2,df.visit1)) \
        .withColumn('diff 2&3',datediff(df.visit3,df.visit2)) 
df = df.withColumn('avg', (df["diff 1&2"]+df["diff 2&3"])/2)

In [ ]:
df.show()

+----------+---------+----------+----------+----------+----------+--------+--------+-----+
|first_name|last_name|       dob|    visit1|    visit2|    visit3|diff 1&2|diff 2&3|  avg|
+----------+---------+----------+----------+----------+----------+--------+--------+-----+
|  Mohammed|   Alfasy|  1987-4-8|2016-07-01|2017-03-02|2018-02-03|     244|     338|291.0|
|     Marcy|Wellmaker|  1986-4-8|2015-07-01|2017-03-01|2018-02-01|     609|     337|473.0|
|     Ginny|   Ginger| 1986-7-10|2014-07-08|2015-03-02|2016-02-03|     237|     338|287.5|
|     Vijay| Doberson|  1988-5-2|2016-07-01|2018-03-02|2018-02-03|     609|     -27|291.0|
|     Orhan|  Gelicek| 1987-5-11|2016-07-05|2017-03-01|2018-02-09|     239|     345|292.0|
|     Sarah|    Jones|  1956-7-6|2016-07-04|2017-03-08|2018-02-10|     247|     339|293.0|
|      John|  Johnson|2017-10-12|2018-02-01|2018-03-10|2018-02-03|      37|     -35|  1.0|
+----------+---------+----------+----------+----------+----------+--------+--------+-----+

In [ ]:
df.select((avg('avg')/30).alias('Avg Waiting months')).show()

+------------------+
|Avg Waiting months|
+------------------+
| 9.183333333333334|
+------------------+



## 8. Can you calculate the age of each patient?

In [ ]:
df = df.withColumn('Age', datediff(current_date(),df.dob)/365)
df.show()

+----------+---------+----------+----------+----------+----------+--------+--------+-----+------------------+
|first_name|last_name|       dob|    visit1|    visit2|    visit3|diff 1&2|diff 2&3|  avg|               Age|
+----------+---------+----------+----------+----------+----------+--------+--------+-----+------------------+
|  Mohammed|   Alfasy|  1987-4-8|2016-07-01|2017-03-02|2018-02-03|     244|     338|291.0|35.868493150684934|
|     Marcy|Wellmaker|  1986-4-8|2015-07-01|2017-03-01|2018-02-01|     609|     337|473.0|36.868493150684934|
|     Ginny|   Ginger| 1986-7-10|2014-07-08|2015-03-02|2016-02-03|     237|     338|287.5| 36.61369863013699|
|     Vijay| Doberson|  1988-5-2|2016-07-01|2018-03-02|2018-02-03|     609|     -27|291.0|              34.8|
|     Orhan|  Gelicek| 1987-5-11|2016-07-05|2017-03-01|2018-02-09|     239|     345|292.0|35.778082191780825|
|     Sarah|    Jones|  1956-7-6|2016-07-04|2017-03-08|2018-02-10|     247|     339|293.0| 66.64383561643835|
|      Joh

## 9. Can you extract the month from the visit columns and call it "Month"?

In [ ]:
df = df.withColumn('month1', month(df.visit1)) \
      .withColumn('month2', month(df.visit2)) \
      .withColumn('month3', month(df.visit3)) 
df.show()

+----------+---------+----------+----------+----------+----------+--------+--------+-----+------------------+------+------+------+
|first_name|last_name|       dob|    visit1|    visit2|    visit3|diff 1&2|diff 2&3|  avg|               Age|month1|month2|month3|
+----------+---------+----------+----------+----------+----------+--------+--------+-----+------------------+------+------+------+
|  Mohammed|   Alfasy|  1987-4-8|2016-07-01|2017-03-02|2018-02-03|     244|     338|291.0|35.868493150684934|     7|     3|     2|
|     Marcy|Wellmaker|  1986-4-8|2015-07-01|2017-03-01|2018-02-01|     609|     337|473.0|36.868493150684934|     7|     3|     2|
|     Ginny|   Ginger| 1986-7-10|2014-07-08|2015-03-02|2016-02-03|     237|     338|287.5| 36.61369863013699|     7|     3|     2|
|     Vijay| Doberson|  1988-5-2|2016-07-01|2018-03-02|2018-02-03|     609|     -27|291.0|              34.8|     7|     3|     2|
|     Orhan|  Gelicek| 1987-5-11|2016-07-05|2017-03-01|2018-02-09|     239|     345

## 10. Challenges with working with date and timestamps

Let's read in the supermarket sales dataframe attached to the lecture now and see some of the issues that can come up when working with date and timestamps values.

## About this dataset

The growth of supermarkets in most populated cities are increasing and market competitions are also high. The dataset is one of the historical sales of supermarket company which has recorded in 3 different branches for 3 months data. 

 - Attribute information
 - Invoice id: Computer generated sales slip invoice identification number
 - Branch: Branch of supercenter (3 branches are available identified by A, B and C).
 - City: Location of supercenters
 - Customer type: Type of customers, recorded by Members for customers using member card and Normal for without member card.
 - Gender: Gender type of customer
 - Product line: General item categorization groups - Electronic accessories, Fashion accessories, Food and beverages, Health and beauty, Home and lifestyle, Sports and travel
 - Unit price: Price of each product in USD
 - Quantity: Number of products purchased by customer
 - Tax: 5% tax fee for customer buying
 - Total: Total price including tax
 - Date: Date of purchase (Record available from January 2019 to March 2019)
 - Time: Purchase time (10am to 9pm)
 - Payment: Payment used by customer for purchase (3 methods are available – Cash, Credit card and Ewallet)
 - COGS: Cost of goods sold
 - Gross margin percentage: Gross margin percentage
 - Gross income: Gross income
 - Rating: Customer stratification rating on their overall shopping experience (On a scale of 1 to 10)

**Source:** https://www.kaggle.com/aungpyaeap/supermarket-sales

### View dataframe and schema as usual

In [ ]:
df = spark.read.csv(path+'supermarket_sales.csv',inferSchema= True, header = True)

In [ ]:
df.show(5)

+-----------+------+---------+-------------+------+--------------------+----------+--------+-------+--------+---------+-------------------+-----------+------+-----------------------+------------+------+
| Invoice ID|Branch|     City|Customer type|Gender|        Product line|Unit price|Quantity| Tax 5%|   Total|     Date|               Time|    Payment|  cogs|gross margin percentage|gross income|Rating|
+-----------+------+---------+-------------+------+--------------------+----------+--------+-------+--------+---------+-------------------+-----------+------+-----------------------+------------+------+
|750-67-8428|     A|   Yangon|       Member|Female|   Health and beauty|     74.69|       7|26.1415|548.9715| 1/5/2019|2023-02-10 13:08:00|    Ewallet|522.83|            4.761904762|     26.1415|   9.1|
|226-31-3081|     C|Naypyitaw|       Normal|Female|Electronic access...|     15.28|       5|   3.82|   80.22| 3/8/2019|2023-02-10 10:29:00|       Cash|  76.4|            4.761904762|      

### Convert date field to date type

Looks like we need to convert the date field into a date type. Let's go ahead and do that..

In [ ]:
df = df.withColumn('Date',to_date(df.Date,'M/d/yyyy'))
df.show()

+-----------+------+---------+-------------+------+--------------------+----------+--------+-------+--------+----------+-------------------+-----------+------+-----------------------+------------+------+
| Invoice ID|Branch|     City|Customer type|Gender|        Product line|Unit price|Quantity| Tax 5%|   Total|      Date|               Time|    Payment|  cogs|gross margin percentage|gross income|Rating|
+-----------+------+---------+-------------+------+--------------------+----------+--------+-------+--------+----------+-------------------+-----------+------+-----------------------+------------+------+
|750-67-8428|     A|   Yangon|       Member|Female|   Health and beauty|     74.69|       7|26.1415|548.9715|2019-01-05|2023-02-10 13:08:00|    Ewallet|522.83|            4.761904762|     26.1415|   9.1|
|226-31-3081|     C|Naypyitaw|       Normal|Female|Electronic access...|     15.28|       5|   3.82|   80.22|2019-03-08|2023-02-10 10:29:00|       Cash|  76.4|            4.761904762| 

### How can we extract the month value from the date field?

If you had trouble converting the date field in the previous question think about a more creative solution to extract the month from that field.

In [ ]:
df.withColumn('Month', month(df.Date)).show()

+-----------+------+---------+-------------+------+--------------------+----------+--------+-------+--------+----------+-------------------+-----------+------+-----------------------+------------+------+-----+
| Invoice ID|Branch|     City|Customer type|Gender|        Product line|Unit price|Quantity| Tax 5%|   Total|      Date|               Time|    Payment|  cogs|gross margin percentage|gross income|Rating|Month|
+-----------+------+---------+-------------+------+--------------------+----------+--------+-------+--------+----------+-------------------+-----------+------+-----------------------+------------+------+-----+
|750-67-8428|     A|   Yangon|       Member|Female|   Health and beauty|     74.69|       7|26.1415|548.9715|2019-01-05|2023-02-10 13:08:00|    Ewallet|522.83|            4.761904762|     26.1415|   9.1|    1|
|226-31-3081|     C|Naypyitaw|       Normal|Female|Electronic access...|     15.28|       5|   3.82|   80.22|2019-03-08|2023-02-10 10:29:00|       Cash|  76.4| 

## 11.0 Working with Arrays

Here is a dataframe of reviews from the movie the Dark Night.

In [ ]:
from pyspark.sql.functions import *

values = [(5,'Epic. This is the best movie I have EVER seen'), \
          (4,'Pretty good, but I would have liked to seen better special effects'), \
          (3,'So so. Casting could have been improved'), \
          (5,'The most EPIC movie of the year! Casting was awesome. Special effects were so intense.'), \
          (4,'Solid but I would have liked to see more of the love story'), \
          (5,'THE BOMB!!!!!!!')]
reviews = spark.createDataFrame(values,['rating', 'review_txt'])

reviews.show(6,False)

+------+--------------------------------------------------------------------------------------+
|rating|review_txt                                                                            |
+------+--------------------------------------------------------------------------------------+
|5     |Epic. This is the best movie I have EVER seen                                         |
|4     |Pretty good, but I would have liked to seen better special effects                    |
|3     |So so. Casting could have been improved                                               |
|5     |The most EPIC movie of the year! Casting was awesome. Special effects were so intense.|
|4     |Solid but I would have liked to see more of the love story                            |
|5     |THE BOMB!!!!!!!                                                                       |
+------+--------------------------------------------------------------------------------------+



## 11.1 Let's see if we can create an array off of the review text column and then derive some meaningful results from it.

**But first** we need to clean the rview_txt column to make sure we can get what we need from our analysis later on. So let's do the following:

1. Remove all punctuation
2. lower case everything
3. Remove white space (trim)
3. Then finally, split the string

In [ ]:
reviews = reviews.withColumn('review_txt', translate(reviews.review_txt, ".,!", ""))
reviews.show(6, False)

+------+-----------------------------------------------------------------------------------+
|rating|review_txt                                                                         |
+------+-----------------------------------------------------------------------------------+
|5     |Epic This is the best movie I have EVER seen                                       |
|4     |Pretty good but I would have liked to seen better special effects                  |
|3     |So so Casting could have been improved                                             |
|5     |The most EPIC movie of the year Casting was awesome Special effects were so intense|
|4     |Solid but I would have liked to see more of the love story                         |
|5     |THE BOMB                                                                           |
+------+-----------------------------------------------------------------------------------+



In [ ]:
reviews = reviews.withColumn('review_txt',lower(reviews.review_txt))
reviews.show(6,False)

+------+-----------------------------------------------------------------------------------+
|rating|review_txt                                                                         |
+------+-----------------------------------------------------------------------------------+
|5     |epic this is the best movie i have ever seen                                       |
|4     |pretty good but i would have liked to seen better special effects                  |
|3     |so so casting could have been improved                                             |
|5     |the most epic movie of the year casting was awesome special effects were so intense|
|4     |solid but i would have liked to see more of the love story                         |
|5     |the bomb                                                                           |
+------+-----------------------------------------------------------------------------------+



In [ ]:
reviews = reviews.withColumn('review_txt', trim(reviews.review_txt))
reviews.show(6,False)

+------+-----------------------------------------------------------------------------------+
|rating|review_txt                                                                         |
+------+-----------------------------------------------------------------------------------+
|5     |epic this is the best movie i have ever seen                                       |
|4     |pretty good but i would have liked to seen better special effects                  |
|3     |so so casting could have been improved                                             |
|5     |the most epic movie of the year casting was awesome special effects were so intense|
|4     |solid but i would have liked to see more of the love story                         |
|5     |the bomb                                                                           |
+------+-----------------------------------------------------------------------------------+



In [ ]:
reviews = reviews.withColumn('review_txt', split(reviews.review_txt,' '))


In [ ]:
reviews.show(6,False)

+------+---------------------------------------------------------------------------------------------------+
|rating|review_txt                                                                                         |
+------+---------------------------------------------------------------------------------------------------+
|5     |[epic, this, is, the, best, movie, i, have, ever, seen]                                            |
|4     |[pretty, good, but, i, would, have, liked, to, seen, better, special, effects]                     |
|3     |[so, so, casting, could, have, been, improved]                                                     |
|5     |[the, most, epic, movie, of, the, year, casting, was, awesome, special, effects, were, so, intense]|
|4     |[solid, but, i, would, have, liked, to, see, more, of, the, love, story]                           |
|5     |[the, bomb]                                                                                        |
+------+-----------

## 11.2 Alright now let's see if we can find which reviews contain the word 'Epic'

In [ ]:
reviews.withColumn('bool',array_contains(reviews.review_txt,"epic")).filter("bool == 'true' ").select('rating','review_txt').toPandas()

,rating,review_txt
0,5,"[epic, this, is, the, best, movie, i, have, ever, seen]"
1,5,"[the, most, epic, movie, of, the, year, casting, was, awesome, special, effects, were, so, intense]"


### That's it! Great Job!